# X-ray absorption spectroscopy with TDDFT
## Full excitation space
## Core-valence separation (CVS) 
## Tamm–Dancoff approximation (TDA)

### VeloxChem in Juputer notebook

In [2]:
import veloxchem as vlx
import numpy as np

np.set_printoptions(precision=4, suppress=True, linewidth=132) # printout format of NumPy arrays

au_to_ang = 0.529177 # length conversion factor
au_to_ev = 27.2114 # energy conversion factor

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


### System specification

In [3]:
mol_str = """
C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, "sto-3g")

### Ordering of atomic orbitals

$\chi_{1s}^{H1}$, $\chi_{1s}^{H2}$, $\chi_{1s}^{H3}$, $\chi_{1s}^{H4}$
$\chi_{1s}^{C1}$, $\chi_{2s}^{C1}$, $\chi_{1s}^{C2}$, $\chi_{2s}^{C2}$
$\chi_{2py}^{C1}$, $\chi_{2py}^{C2}$, $\chi_{2pz}^{C1}$, $\chi_{2pz}^{C2}$, $\chi_{2px}^{C1}$, $\chi_{2px}^{C2}$

### SCF optimization

In [4]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

### Molecular orbitals and energies

In [4]:
F = scf_drv.scf_tensors['E']
C = scf_drv.scf_tensors['C']

print('Orbital energies:\n', F)
print('Coefficients MO 2:\n', C[:,1])
print('Coefficients MO 9:\n', C[:,8])

Orbital energies:
 [-11.0184 -11.0177  -0.975   -0.7497  -0.6087  -0.5238  -0.4729  -0.3183   0.3106   0.6578   0.6952   0.7065   0.9519   0.9593]
Coefficients MO 2:
 [ 0.7014  0.0308 -0.7014 -0.0308 -0.005  -0.005   0.005   0.005   0.      0.      0.      0.     -0.0041 -0.0041]
Coefficients MO 9:
 [ 0.      0.      0.     -0.      0.     -0.     -0.      0.     -0.     -0.      0.8063 -0.8063 -0.      0.    ]


### Ordering of molecular orbitals

Occupied: $1a_{g}$, $1b_{3u}$, $2a_{g}$, $2b_{3u}$, $1b_{2u}$, $3a_{g}$, $1b_{1g}$, $1b_{1u}$

Unoccupied: $1b_{2g}$, $2b_{2u}$, $4a_{g}$, $3b_{3u}$, $4b_{3u}$, $2b_{1g}$

### Core excitation and symmetry
Symmetry of core excited state: $^1 B_{1u}$

Excitation operator: $\hat{\mu}_z$

Electronic excitation channels:

$1b_{3u} \rightarrow 1b_{2g}$

$2b_{3u} \rightarrow 1b_{2g}$

$2b_{3u} \rightarrow 1b_{2g}$

$1b_{1u} \rightarrow 4a_{g}$

Core-excitation channel: 

$1b_{3u} \rightarrow 1b_{2g}$

or with another notation

$\sigma_{u} \rightarrow \pi^{*}$

### Ground to core-excited state transition in a frozen orbital picture
With a single electronic excitation channel:

$\langle X ^1A_g | \hat{\mu}_z | ^1B_{1u} \rangle = \sqrt{2} \langle \sigma_u | \hat{\mu}_z |\pi^* \rangle$

Note: The factor of $\sqrt{2}$ is due to spin-adaptation.

In [5]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()

dipole_mats = dipole_drv.compute(molecule, basis)
z_ao = dipole_mats.z_to_numpy()
z_mo = np.matmul(C.T, np.matmul(z_ao, C))

tmom = np.sqrt(2) * z_mo[1,8]
print(f'Transition moment (a.u.): {tmom : .6f}')
print(f'Orbital energy difference: {F[8]-F[1] : .6f} a.u. (or {(F[8]-F[1]) * au_to_ev : .1f} eV)')

Transition moment (a.u.):  0.139783
Orbital energy difference:  11.328276 a.u. (or  308.3 eV)


## TDDFT framework

Parameterization of SCF states:

\begin{equation}
\label{psi-kappa-param}
  | \bar{\psi}(t) \rangle =
    e^{-i\hat{\kappa}(t)} |0\rangle ; \quad
  \hat{\kappa}(t) =
  \sum_s^\mathrm{unocc} \sum_i^\mathrm{occ} \left[
    \kappa_{si}(t) \hat{a}^\dagger_s \hat{a}_i + \kappa_{si}^*(t) \hat{a}^\dagger_i \hat{a}_s
    \right] ,
\end{equation}

where the creation, $\hat{a}^\dagger$, and annihilation, $\hat{a}$, operators act on unoccupied, secondary, $s$ and occupied, inactive, $i$ molecular orbitals.

The linear response function introduces the RPA matrix

$$
\langle\langle \hat{\mu}_\alpha; \hat{V} \rangle\rangle_\omega =
-{\mu^{[1]}}^\dagger
\Big(
E^{[2]} - \omega S^{[2]}
\Big)^{-1}
V^{[1]}
$$

### Electronic Hessian
In terms of these elementary electronic excitation operators, the electronic Hessian reads

\begin{eqnarray}
E^{[2]} & = &
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	A & B \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	B^* & A^* \\
	\end{pmatrix}
&	= &
    \begin{pmatrix}
    \rule[-10pt]{0pt}{30pt}
    \langle 0_i^s | \hat{H}_0 | 0_j^t \rangle - 
    \langle 0 | \hat{H}_0 | 0 \rangle \delta_{st}\delta_{ij} 
    & 
     -\langle 0_{ij}^{st} | \hat{H}_0 | 0 \rangle 
    (1 - \delta_{ij})(1-\delta_{st}) 
    \\
    \rule[-10pt]{0pt}{30pt}
    -\langle 0 | \hat{H}_0 | 0_{ij}^{st} \rangle 
    (1 - \delta_{ij})(1-\delta_{st}) 
     & 
    \langle 0_j^t | \hat{H}_0 | 0_i^s \rangle - 
    \langle 0 | \hat{H}_0 | 0 \rangle \delta_{st}\delta_{ij} 
    \\
  \end{pmatrix} 
\end{eqnarray}

* The Hessian matrix is Hermitian and has an internal structure in terms of the matrix blocks $A$ and $B$
* The diagonal elements correspond to differences in energy between the single-excited determinants $|0_i^s\rangle$ and the reference state
$|0\rangle$
* The $A$-block corresponds to the CIS approximation

In the Hartree–Fock approximation, the Hessian can be expressed as the differences in orbital energies corrected by an anti-symmetrized two-electron integral

$$
E^{[2]} = 
\begin{pmatrix}
    \rule[-10pt]{0pt}{30pt}
    F_{st} \delta_{ij} \!-\! F_{ji}\delta_{st} \!-\! \langle sj || ti
    \rangle 
    & 
    -\langle st||ij \rangle 
    \\
    \rule[-10pt]{0pt}{30pt}
    -\langle ij||st \rangle 
    & 
    F_{ts} \delta_{ij}\! -\! F_{ij} \delta_{st} 
    \! - \! \langle ti || sj \rangle 
    \\
\end{pmatrix}
$$

* The anti-symmetrized ERI gives the correct asymptotic behavior for charge-transfer excitations
* At the level of DFT, the exchange–correlation functional is introduced and the asymptotic behavior becomes incorrect

#### Spin-adapted excitation operators
In practice, we employ spin-adapted excitation operators

\begin{equation}
  \hat{\kappa} =
  \sum_{s,i} \left[
    \kappa_{si} \hat{E}_{si}^\dagger +
    \kappa_{si}^* \hat{E}_{si}
    \right]
\end{equation}

where the summations run over spatial unoccupied and occupied MOs and the singlet excitation operator reads

$$
\hat{E}_{si}^\dagger = \hat{a}^\dagger_s \hat{a}_i + \hat{a}^\dagger_\bar{s} \hat{a}_\bar{i}
$$

The diagonal elements of the Hessian becomes

$$
E^{[2]}_{si,si} = 2 \times 
\big[
(F_{ss} - F_{ii}) - (ss|ii) + 2(si|is)
\big]
$$

* The factor of 2 stems from the adoption of non-normalized excitation operators
* With triplet excitation operators the exchange integrals vanish

In [6]:
lrs = vlx.LinearResponseEigenSolver()
E2 = lrs.get_e2(molecule, basis, scf_drv.scf_tensors)

nocc = 8
ncore = 2
nvirt = 6
norb = nocc + nvirt

n = nocc * nvirt

print('n =', n)
print('Dimension of E[2] matrix:', E2.shape)
print(f'Diagonal element for (sigma_u, pi*)-core excitation: {E2[6,6] / 2 * au_to_ev : .1f} eV')

# ERI integrals in physicist's notation
moints_drv = vlx.MOIntegralsDriver()
isis = moints_drv.compute_in_mem(molecule, basis, scf_drv.mol_orbs, "OVOV")
iiss = moints_drv.compute_in_mem(molecule, basis, scf_drv.mol_orbs, "OOVV")
print(f'Diagonal element: {(F[8] - F[1] - isis[1,0,1,0] + 2 * iiss[1,1,0,0]) * au_to_ev : .1f} eV')
print(f'Diagonal element without exchange: {(F[8] - F[1] - isis[1,0,1,0]) * au_to_ev : .1f} eV')

* Info * Processing Fock builds... (batch size: 192)                                                                      
* Info *   batch 1/1                                                                                                      
n = 48
Dimension of E[2] matrix: (96, 96)
Diagonal element for (sigma_u, pi*)-core excitation:  292.0 eV
Diagonal element:  292.0 eV
Diagonal element without exchange:  291.2 eV


### Overlap matrix

The overlap matrix in the SCF approximation is trivial and equal to

\begin{eqnarray}
\label{S2}
S^{[2]} & = &
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	1 & 0 \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	0 & -1 \\
	\end{pmatrix} ,
\end{eqnarray}

where the "1" and the "0" are here understood to be the identity and zero matrices, respectively, of the same rank as sub-blocks A and B in the electronic Hessian.

In [7]:
# factor of 2 due to spin-adaptation
S2 = 2 * np.identity(2*n)
S2[n:,n:] *= -1

print('Dimension of S[2] matrix:', S2.shape)

Dimension of S[2] matrix: (96, 96)


### Property gradient

In the SCF approximation, the property gradient takes the form

\begin{equation}
  V^{[1]} =
  \begin{pmatrix}
    \rule[-10pt]{0pt}{20pt}
     \langle 0 | [\hat{a}_{i}^\dagger\hat{a}_{s}, \hat{V}] | 0 \rangle
       \\
    \rule[-5pt]{0pt}{20pt}
     \langle 0 | [\hat{a}_{s}^\dagger\hat{a}_{i}, \hat{V}] | 0 \rangle
\\
  \end{pmatrix} 
  =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
     \langle 0_i^s | \hat{V} | 0 \rangle
       \\
    \rule[-5pt]{0pt}{0pt}
     -\langle 0 | \hat{V} | 0_i^s \rangle
    \\
  \end{pmatrix}
  =
      \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    g
       \\
    \rule[-5pt]{0pt}{0pt}
     -g^*
\\
  \end{pmatrix}
\end{equation}

In [8]:
rsp_drv = vlx.LinearResponseSolver()
V1 = rsp_drv.get_prop_grad('electric dipole', 'z', molecule, basis, scf_drv.scf_tensors)[0]
print('Dimension of V[1] vector:', V1.shape)
print(f'  g: {V1[nvirt] : .6f}')
print(f'-g*: {V1[n + nvirt] : .6f}')
print(f'MO integral 2 * < sigma_u | z | pi* >: {2 * z_mo[1,8] : .6f}')

Dimension of V[1] vector: (96,)
  g:  0.197684
-g*: -0.197684
MO integral 2 * < sigma_u | z | pi* >:  0.197684


### Transition energies
We denote by $X_e$ the eigenvectors of the generalized eigenvalue equation

\begin{equation}
  E^{[2]} X_e = \lambda_e S^{[2]} X_e, \quad e = -n,\ldots,-1,1,\ldots,n ,
\end{equation}
 
where the matrix dimension is $2n$. We find the set of eigenvalues and eigenvectors by diagonalizing the non-Hermitian matrix $\left(S^{[2]}\right)^{-1} E^{[2]}$ according to

\begin{equation}
\label{gen-eig}
  X^{-1} \left[ \left(S^{[2]}\right)^{-1} E^{[2]} \right] X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{9pt}{0pt}
    \lambda & \rule{3pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{3pt}{0pt}
    0 & -\lambda \rule{3pt}{0pt} \\
  \end{bmatrix} ,
\end{equation}

where $\lambda$ is a diagonal matrix of dimension $n$ collecting the eigenvalues with positive index and the columns of $X$ store the eigenvectors ($X$ is assumed to be nonsingular). This pairing of eigenvalues has its correspondence in the eigenvectors through

\begin{equation*}
  X_e =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    Z_e \\
    \rule[-5pt]{0pt}{0pt}
    Y_e^* \\
  \end{pmatrix}
\; \mbox{with eigenvalue}\; \lambda_e
; \qquad
  X_{-e} =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    Y_{e} \\
    \rule[-5pt]{0pt}{0pt}
    Z_{e}^* \\
  \end{pmatrix} 
\; \mbox{with eigenvalue}\; -\!\lambda_e .
\end{equation*}

The matrix $X$ will therefore have the structure

\begin{equation}
\label{X-struct}
  X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{3pt}{0pt}
    Z & \rule{3pt}{0pt} Y \rule{3pt}{0pt} \\
    \rule{8pt}{0pt}
    Y^* & \rule{3pt}{0pt} Z^* \rule{3pt}{0pt} \\
  \end{bmatrix} .
\end{equation}

With an appropriate scaling of the eigenvectors $X_e$, the non-unitary matrix $X$ achieves a simultaneous diagonalization of the two non-commuting matrices $E^{[2]}$ and $S^{[2]}$ as

\begin{equation}
\label{sim-diag}
  X^\dagger E^{[2]} X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{9pt}{0pt}
    \lambda & \rule{3pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{3pt}{0pt}
    0 & \lambda \rule{3pt}{0pt} \\
  \end{bmatrix} ; \qquad
  X^\dagger S^{[2]} X =
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{8pt}{0pt}
    I & \rule{5pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{5pt}{0pt}
    0 & -I \rule{3pt}{0pt} \\
  \end{bmatrix} .
\end{equation}

For each pair of eigenvectors, the second equation involving the metric of the generalized eigenvalue equation defines which of the two that should be indexed with a positive index. We also emphasize that $X^\dagger \neq X^{-1}$ and for this reason the $X_e$'s are eigenvectors neither of $E^{[2]}$ nor of $S^{[2]}$, just as the $\lambda_e$'s are not the eigenvalues of $E^{[2]}$.

In [9]:
eigs, X = np.linalg.eig(np.matmul(np.linalg.inv(S2), E2))
print(eigs)

# the second smallest positive eigenvalue is the sigma-pi* transition
eigs[np.argmin(np.where(eigs>10, eigs, 999))] = 999
pos = np.argmin(np.where(eigs>10, eigs, 999))

Xf = X[:,pos]
Xf =  Xf / np.sqrt(np.matmul(Xf.T,np.matmul(S2,Xf)))

print('Eigenvector position:', pos)
print(f'sigma_u - pi* transition energy: {eigs[pos] : .4f} a.u. (or {eigs[pos] * au_to_ev : .1f} eV)')
print(f'Xf norm under metric S[2] = {np.matmul(Xf.T,np.matmul(S2,Xf)) : .4f}')

[ 10.7286  10.7291 -10.7286 -10.7291  11.5355  11.5355  11.0765 -11.5355 -11.5355 -11.478   11.2445  11.0949  11.0784  11.478
 -11.2445 -11.0784 -11.2449 -11.478  -11.0765 -11.0949  11.0965  11.2449  11.478  -11.0965  -1.5626   1.5626  -1.5137   1.5137
   0.3958  -0.3958  -1.4065  -1.3792   1.4065   1.3792   1.3791  -1.3791   0.3685  -0.3685  -0.6943   1.2962   0.4191   0.6943
   1.3142  -1.3142  -1.2962  -0.4191  -0.5365   0.5365   1.2609  -1.2609   0.598   -0.598   -0.7917  -0.6656   0.6656   0.7917
   1.1817  -0.9081   0.9081  -0.7555  -1.1817   1.1624  -1.166   -1.1624   1.166    0.7555  -1.1062  -1.1474  -1.1382   1.1062
   1.1474   1.1382  -1.0773   1.0773  -1.0282   1.0282  -1.0198   1.0198   1.0059   1.0023  -1.0059   0.8365   0.8396  -0.8245
  -0.8365  -1.0023   0.8245  -0.8396  -0.8772  -0.8717   0.8717   0.8871   0.8772   0.8731  -0.8871  -0.8731]
Eigenvector position: 1
sigma_u - pi* transition energy:  10.7291 a.u. (or  292.0 eV)
Xf norm under metric S[2] =  1.0000


### Transition moments
Since $(\Omega \Lambda)^{-1} = \Lambda^{-1} \Omega^{-1}$, we can form the inverse of the RPA matrix as

\begin{equation}
  \left( E^{[2]} - \omega S^{[2]} \right)^{-1} =
  X \left[ X^\dagger \left( E^{[2]} - \omega S^{[2]} \right) X \right]^{-1} X^\dagger
\end{equation}

which yields an expression for the linear response function that reads

\begin{eqnarray}
\label{lrf-resolved}
  \langle
  \langle
  \hat{\Omega}; \hat{V}
  \rangle
  \rangle & = &
  -
  \left[\Omega^{[1]}\right]^\dagger 
  \sum_{e=1}^n
  \left(
  \frac{
    X_e
    X^\dagger_e
    }{
    \lambda_e - \hbar \omega
    }
  +
  \frac{
    X_e
    X^\dagger_e 
    }{
    \lambda_e + \hbar \omega
    }
  \right) V^{[1]} 
\end{eqnarray}

This equation suggests that we identify the $\lambda$-eigenvalues as excitation energies of the system and transition moments in the SCF approximation are identified as

\begin{equation}
\label{tranmom}
    \langle f | \hat{V} | 0 \rangle =
    X^\dagger_f V^{[1]}
\end{equation}

In [10]:
tmom = np.dot(Xf, V1)
print(f'Transition moment (a.u.): {tmom : .6f}')

Transition moment (a.u.):  0.141356


### Core–valence separation (CVS) approximation

Due to the large energy separation of core and valence electronic states there will be negigiible contribution from the valence-excitaiton channels. 

In [11]:
n = nocc * nvirt
c = ncore * nvirt

E2_CVS = np.zeros((2 * c, 2 * c))

E2_CVS[0:c, 0:c] = E2[0:c, 0:c]
E2_CVS[0:c, c:2*c] = E2[0:c, n:n+c]
E2_CVS[c:2*c, 0:c] = E2[n:n+c, 0:c]
E2_CVS[c:2*c, c:2*c] = E2[n:n+c, n:n+c]

S2_CVS = 2 * np.identity(2*c)
S2_CVS[c:2*c,c:2*c] *= -1

V1_CVS = np.zeros(2*c)
V1_CVS[0:c] = V1[0:c]
V1_CVS[c:2*c] = V1[n:n+c]

eigs_CVS, X_CVS = np.linalg.eig(np.matmul(np.linalg.inv(S2_CVS), E2_CVS))
print(eigs_CVS)

# the second smallest positive eigenvalue is the sigma-pi* transition
eigs_CVS[np.argmin(np.where(eigs_CVS>10, eigs_CVS, 999))] = 999
pos = np.argmin(np.where(eigs_CVS>10, eigs_CVS, 999))

print('Eigenvector position:', pos)
print(f'sigma_u - pi* transition energy in CVS: {eigs_CVS[pos] : .4f} a.u. (or {eigs_CVS[pos] * au_to_ev : .1f} eV)')
Xf_CVS = X_CVS[:,pos]
Xf_CVS =  Xf_CVS / np.sqrt(np.matmul(Xf_CVS.T, np.matmul(S2_CVS, Xf_CVS)))

print(f'Xf norm under metric S[2] = {np.matmul(Xf_CVS.T, np.matmul(S2_CVS, Xf_CVS)) : .4f}')

tmom_CVS = np.dot(Xf_CVS, V1_CVS)
print(f'Transition moment in CVS (a.u.): {tmom_CVS : .6f}')

[ 10.7286  10.7291  11.0764  11.0783  11.0948  11.0964  11.2444  11.2448  11.5355  11.5354  11.478   11.478  -10.7286 -10.7291
 -11.0764 -11.0783 -11.0948 -11.0964 -11.2444 -11.2448 -11.5355 -11.5354 -11.478  -11.478 ]
Eigenvector position: 1
sigma_u - pi* transition energy in CVS:  10.7291 a.u. (or  292.0 eV)
Xf norm under metric S[2] =  1.0000
Transition moment in CVS (a.u.): -0.139695


### Tamm-Dancoff approximation (TDA)

The TDA corresponds to setting $B=0$ in the electronic Hessian and diagonalize matrix block $A$:

\begin{eqnarray}
E^{[2]} & = &
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	A & B \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	B^* & A^* \\
	\end{pmatrix}
\end{eqnarray}

In [12]:
# divide by factors of 2 and sqrt(2) to account for normalization of spin-adapted excitation operators
E2_CVS_TDA = E2_CVS[0:c, 0:c] / 2
V1_CVS_TDA = V1_CVS[0:c] / np.sqrt(2)

eigs_CVS_TDA, X_CVS_TDA = np.linalg.eigh(E2_CVS_TDA)
print(eigs_CVS_TDA)

pos=1
print(f'sigma_u - pi* transition energy with CVS and TDA (eV): {eigs_CVS_TDA[pos] * au_to_ev : .4f}')
Xf_CVS_TDA = X_CVS_TDA[:,pos]

tmom_CVS_TDA = np.dot(Xf_CVS_TDA, V1_CVS_TDA)
print(f'Transition moment with CVS and TDA (a.u.): {tmom_CVS_TDA : .6f}')

[10.7286 10.7291 11.0764 11.0783 11.0948 11.0965 11.2445 11.2449 11.478  11.478  11.5354 11.5355]
sigma_u - pi* transition energy with CVS and TDA (eV):  291.9529
Transition moment with CVS and TDA (a.u.):  0.139783


### Natural transition orbitals (NTOs)

Collecting occupied and unoccupied (real) MOs as vectors

$$
\bar{\phi}_\mathrm{occ} = (\ldots, \phi_i(\mathbf{r}),  \ldots)^T; \quad
\bar{\phi}_\mathrm{unocc} = (\ldots, \phi_s(\mathbf{r}),  \ldots)^T
$$

and scattering the (real) RPA eigenvector into a rectangular transition matrix $T$ of dimension $n_\mathrm{occ} \times n_\mathrm{unocc}$

$$
T_{f, is} = Z_{f, is} - Y_{f, is}
$$

we can write the transition moment as

\begin{equation}
    \langle f | \hat{V} | 0 \rangle =
    X^T_f V^{[1]} = 
    \big(
    Z_f^T, Y_f^T
    \big)
    \begin{pmatrix}
    g\\-g
    \end{pmatrix}
    =
    \big(
    Z_f - Y_f
    \big)^T
    g
    =
    \int
    \hat{\mu}_\alpha
    \bar{\phi}_\mathrm{occ}^T
    T
    \bar{\phi}_\mathrm{unocc} \;
    d^3\mathbf{r}
\end{equation}

By means of a singular value decomposition (SVD), we can factorize the transition matrix

$$
T = U \Lambda V^T
$$

where matrices $U$ and $V$ are unitary and $\Lambda$ is rectangular diagonal.

The SVD transformation matrices define the pairs of hole and electron NTOs

$$
\bar{\phi}_\mathrm{h}^T = \bar{\phi}_\mathrm{occ}^T U; \quad
\bar{\phi}_\mathrm{e} = V^T \bar{\phi}_\mathrm{unocc}
$$

such that

\begin{equation}
    \langle f | \hat{\mu}_\alpha | 0 \rangle =
    \int
    \hat{\mu}_\alpha
    \bar{\phi}_\mathrm{h}^T
    \Lambda
    \bar{\phi}_\mathrm{e} \;
    d^3\mathbf{r}
\end{equation}

In our CVS example, we get 

$$
\bar{\phi}_\mathrm{occ}^T = (\sigma_g, \sigma_u) ; \quad
\Lambda =
\begin{bmatrix}
 \lambda_1 & 0 & 0 & 0 & 0 & 0 \\
 0 & \lambda_2 & 0 & 0 & 0 & 0 \\
\end{bmatrix}; \quad
\bar{\phi}_\mathrm{unocc}^T = (\pi^*, 2b_{2u}, 4a_{g}, 3b_{3u}, 4b_{3u}, 2b_{1g})
$$

and the associated contributing electron orbital becomes

$$
\phi_\mathrm{e} = \sum_s T_{f,is} \phi_s; \quad i=2
$$

In [13]:
Zf = Xf_CVS[:ncore*nvirt]
Yf = Xf_CVS[ncore*nvirt:]
T = np.reshape(Zf - Yf, (ncore,nvirt))
U, L, VT = np.linalg.svd(T)

print('Unitary matrix for occupied MOs:\n', U)
print('Lambda diagonal elements:\n', L)
print('Unitary matrix for unoccupied MOs:\n', VT)

Unitary matrix for occupied MOs:
 [[ 0. -1.]
 [ 1.  0.]]
Lambda diagonal elements:
 [0.7067 0.    ]
Unitary matrix for unoccupied MOs:
 [[-1.     -0.      0.     -0.      0.     -0.    ]
 [-0.      0.0124 -0.9686  0.1776 -0.1157  0.1292]
 [-0.      0.9686  0.0732  0.1699 -0.1107  0.1236]
 [-0.     -0.1776  0.1699  0.9688  0.0203 -0.0227]
 [ 0.      0.1157 -0.1107  0.0203  0.9868  0.0148]
 [-0.     -0.1292  0.1236 -0.0227  0.0148  0.9835]]
